# 1.导入tf.keras

tensorflow2推荐使用keras构建网络，常见的神经网络都包含在`keras.layer`中(最新的`tf.keras`的版本可能和keras不同)

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
print(tf.__version__)
print(tf.keras.__version__)

2.0.0-dev20190713
2.2.4-tf


# 2.构建简单模型

## 2.1模型堆叠
最常见的模型类型是层的堆叠：`tf.keras.Sequential` 模型

In [2]:
model = tf.keras.Sequential()
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

## 2.2网络配置
`tf.keras.layers`中网络配置：

- `activation`：设置层的激活函数。此参数由内置函数的名称指定，或指定为可调用对象。默认情况下，系统不会应用任何激活函数。

- `kernel_initializer` 和 `bias_initializer`：创建层权重（核和偏差）的初始化方案。此参数是一个名称或可调用对象，默认为 “Glorot uniform” 初始化器。

- `kernel_regularizer` 和 `bias_regularizer`：应用层权重（核和偏差）的正则化方案，例如 L1 或 L2 正则化。默认情况下，系统不会应用正则化函数。


In [3]:
layers.Dense(32, activation='sigmoid')
layers.Dense(32, activation=tf.sigmoid)
layers.Dense(32, kernel_initializer='orthogonal')
layers.Dense(32, kernel_initializer=tf.keras.initializers.glorot_normal)
layers.Dense(32, kernel_regularizer=tf.keras.regularizers.l2(0.01))
layers.Dense(32, kernel_regularizer=tf.keras.regularizers.l1(0.01))

# 3.训练和评估

## 3.1设置训练流程
构建好模型后，通过调用 `compile` 方法配置该模型的学习流程：

In [4]:
model = tf.keras.Sequential()
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=[tf.keras.metrics.categorical_accuracy])

## 3.2输入Numpy数据

In [24]:
import numpy as np

train_x = np.random.random((1000, 72))
train_y = np.random.random((1000, 10))

val_x = np.random.random((200, 72))
val_y = np.random.random((200, 10))

model.fit(train_x, train_y, epochs=10, batch_size=100,
          validation_data=(val_x, val_y))

Train on 1000 samples, validate on 200 samples
Epoch 1/10
1000/1000 [==============================] - 0s 238us/sample - loss: 11.4386 - accuracy: 0.1000 - val_loss: 11.5589 - val_accuracy: 0.0850
Epoch 2/10
1000/1000 [==============================] - 0s 23us/sample - loss: 11.4370 - accuracy: 0.1040 - val_loss: 11.5583 - val_accuracy: 0.1000
Epoch 3/10
1000/1000 [==============================] - 0s 23us/sample - loss: 11.4361 - accuracy: 0.1040 - val_loss: 11.5581 - val_accuracy: 0.1050
Epoch 4/10
1000/1000 [==============================] - 0s 24us/sample - loss: 11.4352 - accuracy: 0.1010 - val_loss: 11.5580 - val_accuracy: 0.1000
Epoch 5/10
1000/1000 [==============================] - 0s 26us/sample - loss: 11.4347 - accuracy: 0.1020 - val_loss: 11.5578 - val_accuracy: 0.1000
Epoch 6/10
1000/1000 [==============================] - 0s 24us/sample - loss: 11.4342 - accuracy: 0.1060 - val_loss: 11.5578 - val_accuracy: 0.1150
Epoch 7/10
1000/1000 [==============================] - 0s

## 3.3tf.data输入数据

In [25]:
dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
dataset = dataset.batch(32)
dataset = dataset.repeat()
val_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y))
val_dataset = val_dataset.batch(32)
val_dataset = val_dataset.repeat()

model.fit(dataset, epochs=10, steps_per_epoch=30,
          validation_data=val_dataset, validation_steps=3)

W0715 11:59:59.142257 140035963959104 training_utils.py:1461] Expected a shuffled dataset but input dataset `x` is not shuffled. Please invoke `shuffle()` on input dataset.


Epoch 1/10
30/30 [==============================] - 0s 6ms/step - loss: 11.4280 - accuracy: 0.1042 - val_loss: 11.5910 - val_accuracy: 0.1250
Epoch 2/10
30/30 [==============================] - 0s 3ms/step - loss: 11.4438 - accuracy: 0.1175 - val_loss: 11.5917 - val_accuracy: 0.1458
Epoch 3/10
30/30 [==============================] - 0s 4ms/step - loss: 11.4182 - accuracy: 0.1111 - val_loss: 11.5914 - val_accuracy: 0.0833
Epoch 4/10
30/30 [==============================] - 0s 4ms/step - loss: 11.4504 - accuracy: 0.1250 - val_loss: 11.5914 - val_accuracy: 0.1146
Epoch 5/10
30/30 [==============================] - 0s 4ms/step - loss: 11.4266 - accuracy: 0.1271 - val_loss: 11.5921 - val_accuracy: 0.1146
Epoch 6/10
30/30 [==============================] - 0s 4ms/step - loss: 11.4447 - accuracy: 0.1239 - val_loss: 11.5922 - val_accuracy: 0.1250
Epoch 7/10
30/30 [==============================] - 0s 4ms/step - loss: 11.4286 - accuracy: 0.1218 - val_loss: 11.5922 - val_accuracy: 0.1042
Epoch 

## 3.4评估与预测

In [21]:
test_x = np.random.random((1000, 72))
test_y = np.random.random((1000, 10))
model.evaluate(test_x, test_y, batch_size=32) # evaluate 评估函数，返回 loss 和 模型指标
test_data = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_data = test_data.batch(32).repeat()
model.evaluate(test_data, steps=30)

30/30 [==============================] - 0s 2ms/step - loss: 11.4748 - categorical_accuracy: 0.1219


[11.474751154581705, 0.121875]

In [22]:
# predict
result = model.predict(test_x, batch_size=32)
print(result)

[[0.10304388 0.09661334 0.09357048 ... 0.08873349 0.08596982 0.11420347]
 [0.10955941 0.09514632 0.0937947  ... 0.09728017 0.09583507 0.10026734]
 [0.08722668 0.10540339 0.09237859 ... 0.09828128 0.10393666 0.11418623]
 ...
 [0.09711259 0.09632509 0.10145363 ... 0.10295092 0.10708995 0.10351047]
 [0.09467735 0.10575885 0.09388798 ... 0.09833457 0.09909901 0.11519182]
 [0.09708747 0.0987007  0.09871585 ... 0.1091151  0.1012155  0.11196709]]


# 4.构建高级模型

## 4.1函数式api
`tf.keras.Sequential` 模型是层的简单堆叠，无法表示任意模型。使用 Keras 函数式 API 可以构建复杂的模型拓扑，例如：

- 多输入模型

- 多输出模型

- 具有共享层的模型（同一层被调用多次）

- 具有非序列数据流的模型（例如，残差连接）

**使用函数式 API 构建的模型具有以下特征：**

层实例可调用并返回张量。
输入张量和输出张量用于定义 `tf.keras.Model` 实例。
此模型的训练方式和 Sequential 模型一样。


In [23]:
input_x = tf.keras.Input(shape=(72,))
hidden1 = layers.Dense(32, activation='relu')(input_x)
hidden2 = layers.Dense(16, activation='relu')(hidden1)
pred = layers.Dense(10, activation='softmax')(hidden2)

model = tf.keras.Model(inputs=input_x, outputs=pred)
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=32, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 330us/sample - loss: 11.6665 - accuracy: 0.1050
Epoch 2/5
1000/1000 [==============================] - 0s 74us/sample - loss: 11.6285 - accuracy: 0.0930
Epoch 3/5
1000/1000 [==============================] - 0s 73us/sample - loss: 11.6258 - accuracy: 0.1020
Epoch 4/5
1000/1000 [==============================] - 0s 80us/sample - loss: 11.6250 - accuracy: 0.1090
Epoch 5/5
1000/1000 [==============================] - 0s 85us/sample - loss: 11.6237 - accuracy: 0.1170


## 4.2模型子类化
通过对 `tf.keras.Model` 进行子类化并定义您自己的前向传播来构建完全可自定义的模型。在 `__init__` 方法中创建层并将它们设置为类实例的属性。在 `call` 方法中定义前向传播

In [26]:
class MyModel(tf.keras.Model):
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        self.layer1 = layers.Dense(32, activation='relu')
        self.layer2 = layers.Dense(num_classes, activation='softmax')
    def call(self, inputs):
        h1 = self.layer1(inputs)
        out = self.layer2(h1)
        return out
    
    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.num_classes
        return tf.TensorShape(shape)

model = MyModel(num_classes=10)
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])

model.fit(train_x, train_y, batch_size=16, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 303us/sample - loss: 11.5736 - accuracy: 0.1030
Epoch 2/5
1000/1000 [==============================] - 0s 88us/sample - loss: 11.4701 - accuracy: 0.1000
Epoch 3/5
1000/1000 [==============================] - 0s 93us/sample - loss: 11.4434 - accuracy: 0.1080
Epoch 4/5
1000/1000 [==============================] - 0s 77us/sample - loss: 11.4341 - accuracy: 0.1130
Epoch 5/5
1000/1000 [==============================] - 0s 127us/sample - loss: 11.4306 - accuracy: 0.1030


## 4.3自定义层
通过对 `tf.keras.layers.Layer` 进行子类化并实现以下方法来创建自定义层：

- `build`：创建层的权重。使用 `add_weight` 方法添加权重。

- `call`：定义前向传播。

- `compute_output_shape`：指定在给定输入形状的情况下如何计算层的输出形状。
或者，可以通过实现 `get_config` 方法和 `from_config` 类方法序列化层。


In [30]:
class MyLayer(layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)
    
    def build(self, input_shape):
        shape = tf.TensorShape((input_shape[1], self.output_dim))
        self.kernel = self.add_weight(name='kernel1', shape=shape,
                                   initializer='uniform', trainable=True)
        super(MyLayer, self).build(input_shape)
    
    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)

    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.output_dim
        return tf.TensorShape(shape)

    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.output_dim
        return base_config

    @classmethod
    def from_config(cls, config):
        return cls(**config)
    
model = tf.keras.Sequential(
[
    MyLayer(10),
    layers.Activation('softmax')
])


model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])

model.fit(train_x, train_y, batch_size=16, epochs=5)


Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 223us/sample - loss: 11.4534 - accuracy: 0.1160
Epoch 2/5
1000/1000 [==============================] - 0s 90us/sample - loss: 11.4448 - accuracy: 0.1000
Epoch 3/5
1000/1000 [==============================] - 0s 83us/sample - loss: 11.4367 - accuracy: 0.1350
Epoch 4/5
1000/1000 [==============================] - 0s 75us/sample - loss: 11.4320 - accuracy: 0.1350
Epoch 5/5
1000/1000 [==============================] - 0s 92us/sample - loss: 11.4276 - accuracy: 0.1380


## 4.3回调

In [31]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(train_x, train_y, batch_size=16, epochs=5,
         callbacks=callbacks, validation_data=(val_x, val_y))

Train on 1000 samples, validate on 200 samples
Epoch 1/5
1000/1000 [==============================] - 0s 163us/sample - loss: 11.4211 - accuracy: 0.1360 - val_loss: 11.5647 - val_accuracy: 0.0850
Epoch 2/5
1000/1000 [==============================] - 0s 109us/sample - loss: 11.4195 - accuracy: 0.1490 - val_loss: 11.5751 - val_accuracy: 0.1100
Epoch 3/5
1000/1000 [==============================] - 0s 109us/sample - loss: 11.4162 - accuracy: 0.1460 - val_loss: 11.5723 - val_accuracy: 0.0900


# 5.保持和恢复

## 5.1权重保存

In [32]:
model.save_weights('./weights/model')
model.load_weights('./weights/model')
model.save_weights('./model.h5')
model.load_weights('./model.h5')

## 5.3保存整个模型

In [36]:
model = tf.keras.Sequential([
  layers.Dense(10, activation='softmax', input_shape=(72,)),
  layers.Dense(10, activation='softmax')
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=32, epochs=5)
model.save('all_model.h5')
model = tf.keras.models.load_model('all_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 336us/sample - loss: 11.4460 - accuracy: 0.0980
Epoch 2/5
1000/1000 [==============================] - 0s 70us/sample - loss: 11.4390 - accuracy: 0.0930
Epoch 3/5
1000/1000 [==============================] - 0s 72us/sample - loss: 11.4370 - accuracy: 0.0980
Epoch 4/5
1000/1000 [==============================] - 0s 71us/sample - loss: 11.4359 - accuracy: 0.0950
Epoch 5/5
1000/1000 [==============================] - 0s 78us/sample - loss: 11.4356 - accuracy: 0.0810
